In [10]:
import sys
import io
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager


# Configure Chrome options to bypass detection
chrome_options = Options()
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# Initialize ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to scrape job details
def scrape_indeed_jobs(base_url):
    driver.get(base_url)
    sleep(2)  # Allow page to load

    all_jobs = []  # List to hold all job data
    total_jobs = 0
    page = 1
    
    while True:
        print(f"Scraping page {page}...")
        jobs = driver.find_elements(By.CSS_SELECTOR, ".job_seen_beacon")  # Select each job posting element

        i = 0
        for job in jobs:
            job_data = {}  # Dictionary to hold a single job's details
            i = i + 1
            try:
                # Job title (clickable element)
                try:
                    title_element = job.find_element(By.XPATH, f"/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a")
                    job_data['title'] = title_element.text
                except:
                    job_data['title'] = "No title found"

                # Company name
                try:
                    job_data['company'] = job.find_element(By.XPATH, f"/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[2]/div/div[1]/span").text
                except:
                    job_data['company'] = "No company name found"

                # Location
                try:
                    job_data['location'] = job.find_element(By.XPATH, f"/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[2]/div/div[2]").text
                except:
                    job_data['location'] = "No location found"

                # Click the job title to navigate to the detailed job page
                title_element.click()
                sleep(5)  # Wait for the page to load

                # Scrape additional information from the detailed job page
                # Company rating (if exists)
                try:
                    job_data['rating'] = driver.find_element(By.XPATH, f"/html/body/main/div/div[2]/div/div[5]/div/div[2]/div/div/div[2]/div[2]/div[1]/div/div[1]/div[2]/div/div/div/div[1]/div[2]/span[1]").text
                except:
                    job_data['rating'] = 0

                # Full job description
                try:
                    job_data['description'] = driver.find_element(By.ID, "jobDescriptionText").text
                except:
                    job_data['description'] = "No description found"

                # Add job data to list
                all_jobs.append(job_data)

                print(f"Scraped job {total_jobs + 1}: {job_data}")
                print("-" * 50)

                total_jobs += 1

            except Exception as e:
                print(f"Error scraping job: {e}")

    # Save all job data to a JSON file
    with open("scraped_jobs.json", "w", encoding="utf-8") as f:
        json.dump(all_jobs, f, ensure_ascii=False, indent=4)

    print(f"Scraping completed. Total jobs found: {total_jobs}")
    driver.quit()

# URL for Indeed job search for Data Scientist in the USA

for page in range(0, 100, 10):
    base_url = f"https://www.indeed.com/jobs?q=data+scientist&l=usa&start={page}"
    scrape_indeed_jobs(base_url)


Scraping page 1...
Scraped job 1: {'title': 'Data Scientist', 'company': 'SAS', 'location': 'Remote in Cary, NC 27513', 'rating': '4.1', 'description': ''}
--------------------------------------------------
Scraped job 2: {'title': '2025 Cat Digital Early Career Data Scientist Opportunities', 'company': 'Caterpillar', 'location': 'Peoria, IL', 'rating': 0, 'description': "Career Area:\nBusiness Technologies, Digital and Data\nJob Description:\nYour Work Shapes the World at Caterpillar Inc.\nWhen you join Caterpillar, you're joining a global team who cares not just about the work we do – but also about each other. We are the makers, problem solvers, and future world builders who are creating stronger, more sustainable communities. We don't just talk about progress and innovation here – we make it happen, with our customers, where we work and live. Together, we are building a better world, so we can all enjoy living in it.\nYour Work Shapes the World at Caterpillar Inc.\nWhen you join Ca

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.90)
Stacktrace:
	GetHandleVerifier [0x006A7143+25587]
	(No symbol) [0x0063A2E4]
	(No symbol) [0x00532113]
	(No symbol) [0x0050E23B]
	(No symbol) [0x005A179F]
	(No symbol) [0x005B4CB9]
	(No symbol) [0x0059A936]
	(No symbol) [0x0056BA73]
	(No symbol) [0x0056C4CD]
	GetHandleVerifier [0x00984C63+3030803]
	GetHandleVerifier [0x009D6B99+3366473]
	GetHandleVerifier [0x007395F2+624802]
	GetHandleVerifier [0x00740E6C+655644]
	(No symbol) [0x00642C9D]
	(No symbol) [0x0063FD68]
	(No symbol) [0x0063FF05]
	(No symbol) [0x00632336]
	BaseThreadInitThunk [0x75B8FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771380CE+286]
	RtlGetAppContainerNamedObjectPath [0x7713809E+238]
